<a href="https://colab.research.google.com/github/mantasbandonis/denoiser/blob/main/denoiser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/mantasbandonis/denoiser

Cloning into 'denoiser'...
remote: Enumerating objects: 13422, done.
remote: Counting objects: 100% (413/413), done.
remote: Compressing objects: 100% (240/240), done.
remote: Total 13422 (delta 163), reused 381 (delta 147), pack-reused 13009
Receiving objects: 100% (13422/13422), 287.51 MiB | 30.42 MiB/s, done.
Resolving deltas: 100% (164/164), done.
Checking out files: 100% (13041/13041), done.


In [9]:
import os
os.getcwd()

'/content/denoiser'

In [2]:
%cd /content/denoiser

/content/denoiser


In [16]:
#%cd /content/denoiser/dataset/debug

/content/denoiser/dataset/debug


In [4]:
!pip install -r requirements.txt

  Cloning https://github.com/ludlows/python-pesq to /tmp/pip-install-knditlb4/pesq_1207e4f9b2df43ab8b0e9f03064058ba
  Running command git clone -q https://github.com/ludlows/python-pesq /tmp/pip-install-knditlb4/pesq_1207e4f9b2df43ab8b0e9f03064058ba
     |████████████████████████████████| 72 kB 541 kB/s 
     |████████████████████████████████| 58 kB 7.2 MB/s 
     |████████████████████████████████| 14.6 MB 33.3 MB/s 
     |████████████████████████████████| 3.2 MB 32.1 MB/s 
     |████████████████████████████████| 753.2 MB 14 kB/s 
     |████████████████████████████████| 58 kB 7.1 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 41 kB 680 kB/s 
     |████████████████████████████████| 41 kB 181 kB/s 
     |████████████████████████████████| 40 kB 6.5 MB/s 
     |████████████████████████████████| 40 kB 6.3 MB/s 
  Created wheel for pesq: filename=pesq-0.0.3-cp37-cp37m-linux_x86_64.whl size=211368 sha256=3d38447b5c95d07e90b3b129c5680aef6c

In [12]:
%mkdir denoiser/dataset/debug/out

In [18]:
!pip install denoiser

     |████████████████████████████████| 48 kB 4.1 MB/s 
  Created wheel for denoiser: filename=denoiser-0.1.4-py3-none-any.whl size=42782 sha256=c6865ce1097fe04c740962b4dfd2863dd04c7945680dcf2f9537c4cd306319f3
  Stored in directory: /root/.cache/pip/wheels/76/d9/3d/f34ed4201f8090d6c2adf117ba05acec0c4635d0ed7416e0d9
Successfully built denoiser


In [5]:
!python -m denoiser.audio

[]

In [7]:
!pip install pycocotools==2.0.0

     |████████████████████████████████| 1.5 MB 5.1 MB/s 
  Created wheel for pycocotools: filename=pycocotools-2.0.0-cp37-cp37m-linux_x86_64.whl size=264716 sha256=75e80207db0ef49cc6bdb5c8eec98e3c4885f9e16f35798e5876e9821aa88117
  Stored in directory: /root/.cache/pip/wheels/4d/50/dc/e1f07e9eb5678a0ee21bc091220f1f3806ba8e48ef3f2083cb
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.4
    Uninstalling pycocotools-2.0.4:
      Successfully uninstalled pycocotools-2.0.4


In [13]:
%mkdir egs/debug/tr
!python -m denoiser.audio dataset/debug/noisy > egs/debug/tr/noisy.json
!python -m denoiser.audio dataset/debug/clean > egs/debug/tr/clean.json

In [14]:
! ./train.py

[2022-04-28 20:35:30,862][__main__][INFO] - For logs, checkpoints and samples check /content/denoiser/outputs/exp_
[2022-04-28 20:35:31,631][__main__][ERROR] - Some error happened
Traceback (most recent call last):
  File "./train.py", line 105, in main
    _main(args)
  File "./train.py", line 99, in _main
    run(args)
  File "./train.py", line 53, in run
    args.dset.train, length=length, stride=stride, pad=args.pad, **kwargs)
  File "/content/denoiser/denoiser/data.py", line 93, in __init__
    assert len(self.clean_set) == len(self.noisy_set)
AssertionError
